In [1]:
import pandas as pd
from math import isnan

In [2]:
GT_plates = pd.read_excel("PlatesLateral.ods", engine="odf")
results = pd.read_csv('../out/results.csv')
GT_plates = GT_plates[GT_plates['Plate'].notna()]
GT_plates

,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,N
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,N
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,N
134,w3,EHS,“0718”,NaN,NaN,N
135,w3,Rare,SN-33XG,NaN,NaN,Y


In [3]:
results

,Model,Video,Frame,Pred_plate,Confidence
0,a109,a109e_takeoff,0,NaN,0.000000
1,a109,a109e_takeoff,1,JASO1,0.921875
2,a109,a109e_takeoff,2,JASRMN,0.890625
3,a109,a109e_takeoff,3,JASROM,0.890625
4,a109,a109e_takeoff,4,JADBMN,0.906250
...,...,...,...,...,...
4097,w3,Saving,102,NaN,0.000000
4098,w3,Saving,103,NaN,0.000000
4099,w3,Saving,104,NaN,0.000000
4100,w3,Saving,105,NaN,0.000000


In [4]:
n = GT_plates.shape[0]
plates_found = 0
plates_found_by_voting = 0
detected = []
for i,row in GT_plates.iterrows():
    #Get plates recognized in that video
    preds = results.loc[(results['Model'] == row['Model']) & (results['Video'] == row['Video'])]
    #If a frame interval is defined, get only plates in that intervals
    if not isnan(row['Init_frame']):
        preds = preds[preds['Frame'].isin(range(int(row['Init_frame']),int(row['End_frame']+1)))]
    #Check if plate was found
    print(row['Plate'])
    print(list(preds['Pred_plate']))
    if row['Plate'] in list(preds['Pred_plate']) or str(row['Plate']).replace('-','') in list(preds['Pred_plate']):
        plates_found+=1
        detected+=['Y']
    else:
        detected+=['N']
    if len(preds['Pred_plate'].mode()):
        democracy_plate = preds['Pred_plate'].mode()[0]
        if democracy_plate == row['Plate']:
            plates_found_by_voting+=1
GT_plates['Detected'] = detected
GT_plates.to_excel("PlatesLateral.ods", engine="odf",index=False)
print(plates_found/n,plates_found_by_voting/n)
GT_plates

JA6801
[nan, 'JASO1', 'JASRMN', 'JASROM', 'JADBMN', 'JASBO', 'JASEM', 'JASO1', 'JASOM', 'JAbBMM', 'JA6B801', 'JBUSR', 'JA68O1', 'JASBO1', 'JABBOI', 'JAB8O1', 'JA6BO1', 'JA6BOT', 'JABBO1', 'JASRON']
D-HAVN
['D-HAVN', 'D-HAVN', 'D-HAVN', 'D-HAVN', 'D-HAVN', 'D-HANN', 'D-HANN', 'D-HANN', nan, 'D-HANITS', 'D-HANTS', nan, nan]
S2-AJP
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'S2-AE', 'S2-ANP', 'S2-AUP', nan, nan, nan, nan, nan, nan, 'S2-AJP', 'MLAIE', '2-AJP', nan, 'S2-AJP', 'S2-AJP', 'S2-AJP', 'S2-AJP']
N433AK
['NA3BAK', 'N33AK', 'NA38AK', 'NB3A1K', 'NLRBAK', nan, 'C-AORN', 'NA3BAK', 'NBBAK', 'N433AK', 'NAB3AK', 'NARBAK', nan, nan, 'MOBAr', 'MBAK', nan, nan, 'MAZBAK', nan, nan, nan, nan, nan, nan, 'NSEAK', nan]
N109EX
[nan, 'NTO9EX', nan, 'N1O9EX', 'N1OBEX', 'N1O9EX', 'N1O9EX', nan]
PP-JMA
[nan, nan, 'PP-JMA', nan, nan, nan]
PP-MMC
[nan, nan, nan, nan, nan, nan]
PT-YKK
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

F-WGYP
[nan]
VH-IXX
[nan]
FMP
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
LY-HTB
[nan, nan, nan, nan, 'LY-HT8', 'LY-HTB', nan, 'LY-hia', nan, nan, nan, nan, nan, 'JeRMS', 'LY-HTB', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'LY-HTB', nan, nan, 'LY-HTB']
EC-MTG
['NORLDAAT', nan, 'MGRDAHATON', 'HLAMTG', 'EC-MTG', 'EC-MTG', 'EC-MTG', 'MORLDAVATION', 'EC-MTe', 'MONDASATON', 'MOLO', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'MONCBATON', 'MORBARASTON', nan]
N130CS
['N30CS', 'NBOCS', 'NBOCS', 'NBOCS', 'NBOCS', 'NBOCS', 'NBOCS', 'NB3OCS', 'N30CS', 'NBOCS', 'NBOCS', 'N30CS', 'NBOCS', 'NBOCS', nan, 'NRANS', nan, nan, 'N20C', nan, nan, nan, nan, nan, nan, nan, nan, 'NIROCS', 'MVOCS', nan, nan]
8257
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

,Model,Video,Plate,Init_frame,End_frame,Detected
0,a109,a109e_takeoff,JA6801,NaN,NaN,N
1,a109,augusta_a109,D-HAVN,NaN,NaN,Y
2,a109,aw109sp,S2-AJP,NaN,NaN,Y
3,a109,corporate,N433AK,NaN,NaN,Y
4,a109,crashed,N109EX,NaN,NaN,N
...,...,...,...,...,...,...
132,uh1,huey,VH-UYH,57.0,113.0,Y
133,uh1,uh1,N617WC,NaN,NaN,N
134,w3,EHS,“0718”,NaN,NaN,N
135,w3,Rare,SN-33XG,NaN,NaN,Y
